In [1]:
!pip install category_encoders

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from google.colab import drive
from category_encoders import TargetEncoder
import xgboost as xgb

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

# Load datasets
train_path = '/content/drive/MyDrive/DATASET/train.csv'
test_path = '/content/drive/MyDrive/DATASET/test_x.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-3-aafee6fd9931>:7: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_path)


In [4]:
train_df.head()

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,...,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,...,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4


In [5]:
train_df.describe(include='all')

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
count,65125.000000,65124.000000,64956,64948,64334,63088,64993,64870,22440.000000,65125,...,32850,52283,15607,36817,52285,14599,58671,28592,419,65125.000000
unique,NaN,NaN,4,13498,2230,319,527,4,NaN,5,...,9,2,9490,2,2,9286,2,4,168,NaN
top,NaN,NaN,Kadın,1/1/70 2:00,İstanbul,İstanbul,İSTANBUL ÜNİVERSİTESİ,Devlet,NaN,Hayır,...,0,Hayır,-,Hayır,Hayır,-,Evet,Orta,Anadolu Üniversitesi,NaN
freq,NaN,NaN,32077,1655,8682,16625,2083,37885,NaN,51978,...,11315,37716,2956,19661,33919,5196,48872,13057,26,NaN
mean,2018.277298,32.086466,NaN,NaN,NaN,NaN,NaN,NaN,34.226181,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32562.000000
std,2.669979,18.139239,NaN,NaN,NaN,NaN,NaN,NaN,41.632412,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18800.112478
min,2014.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,2016.000000,18.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16281.000000
50%,2019.000000,29.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32562.000000
75%,2021.000000,44.000000,NaN,NaN,NaN,NaN,NaN,NaN,75.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48843.000000


In [6]:
# Check for missing values
train_df.isnull().sum().sort_values(ascending=True)

,0
Basvuru Yili,0
Burs Aliyor mu?,0
id,0
Degerlendirme Puani,1
Universite Adi,132
Cinsiyet,169
Dogum Tarihi,177
Bölüm,231
Baska Bir Kurumdan Burs Aliyor mu?,231
Universite Turu,255


In [7]:
# Set the threshold and drop the columns with more than 40k missing value
missing_value_threshold = 42000
train_df = train_df.loc[:, train_df.isnull().sum() < missing_value_threshold]

print(train_df.shape)

(65125, 35)


In [8]:
train_df = train_df.dropna(subset=['Degerlendirme Puani'])  # Delete missing 'Degerlendirme Puani' value

In [9]:
# Preprocessing: Lowercasing and transliteration function
def transliterate_turkish(text):
    if pd.isna(text):
        return text     # Return NaN without modification
    if not isinstance(text, str):
        return text     # Return non-string values without modification
    turkish_to_english = str.maketrans("üÜıIİğĞşŞçÇöÖ", "uUiIigGsScCoO")
    text = text.translate(turkish_to_english).lower().strip()
    return text

# Apply on all columns
str_cols = train_df.select_dtypes(include=['object']).columns
for col in str_cols:
    train_df[col] = train_df[col].apply(transliterate_turkish)

In [10]:
train_df['Cinsiyet'] = train_df['Cinsiyet'].map({'erkek': 1, 'kadin': 0})

# Replace missing values with random
missing_mask = train_df['Cinsiyet'].isnull()
train_df.loc[missing_mask, 'Cinsiyet'] = np.random.choice([0, 1], size=missing_mask.sum())

binary_cols = [
    'Burs Aliyor mu?',
    'Baska Bir Kurumdan Burs Aliyor mu?',
    'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
    'Profesyonel Bir Spor Daliyla Mesgul musunuz?',
    'Aktif olarak bir STK üyesi misiniz?',
    'Girisimcilikle Ilgili Deneyiminiz Var Mi?',
    'Ingilizce Biliyor musunuz?',
    'Stk Projesine Katildiniz Mi?',
    'Daha Once Baska Bir Universiteden Mezun Olmus'
]
for col in binary_cols:
    # 1 to evet and 0 to hayir
    train_df[col] = train_df[col].map({'evet': 1, 'hayir': 0})

    # replace missing values with random
    missing_mask = train_df[col].isnull()
    train_df.loc[missing_mask, col] = np.random.choice([0, 1], size=missing_mask.sum())

# convert to binary 1 to devet and 0 to ozel
train_df['Universite Turu'] = train_df['Universite Turu'].map({'devlet': 1, 'ozel': 0})
missing_mask = train_df['Universite Turu'].isnull()
train_df.loc[missing_mask, 'Universite Turu'] = np.random.choice([0, 1], size=missing_mask.sum())

# Check columns
print(train_df[['Cinsiyet'] + binary_cols + ['Universite Turu']].head())

   Cinsiyet  Burs Aliyor mu?  Baska Bir Kurumdan Burs Aliyor mu?  \
0       1.0                1                                 0.0   
1       1.0                0                                 0.0   
2       1.0                0                                 0.0   
3       1.0                1                                 0.0   
4       1.0                1                                 0.0   

   Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?  \
0                                                1.0      
1                                                0.0      
2                                                1.0      
3                                                1.0      
4                                                0.0      

   Profesyonel Bir Spor Daliyla Mesgul musunuz?  \
0                                           1.0   
1                                           0.0   
2                                           0.0   
3                               

In [11]:
train_df["Profesyonel Bir Spor Daliyla Mesgul musunuz?"].value_counts()

,count
Profesyonel Bir Spor Daliyla Mesgul musunuz?,
0.0,42799
1.0,22325


In [12]:
import re

# List of Turkish cities
cities = [
    "adana", "adiyaman", "afyonkarahisar", "agri", "aksaray", "amasya", "ankara", "antalya",
    "ardahan", "artvin", "aydin", "balikesir", "bartin", "batman", "bayburt", "bilecik", "bingol",
    "bitlis", "bolu", "burdur", "bursa", "canakkale", "cankiri", "corum", "denizli", "diyarbakir",
    "duzce", "edirne", "elazig", "erzincan", "erzurum", "eskisehir", "gaziantep", "giresun", "gumushane",
    "hakkari", "hatay", "igdir", "isparta", "istanbul", "izmir", "kahramanmaras", "karabuk", "karaman",
    "kars", "kastamonu", "kayseri", "kilis", "kirikkale", "kirklareli", "kirsehir", "kocaeli", "konya",
    "kutahya", "malatya", "manisa", "mardin", "mersin", "mugla", "mus", "nevsehir", "nigde", "ordu",
    "osmaniye", "rize", "sakarya", "samsun", "siirt", "sinop", "sivas", "sanliurfa", "sirnak", "tekirdag",
    "tokat", "trabzon", "tunceli", "usak", "van", "yalova", "yozgat", "zonguldak"
]

# Function for processing city-related columns
def process_city_column(text):
    # Split by '/' and extract the last part
    city = text.split('/')[-1].strip()

    if city in cities:
        return city
    else:
        return text

for col in ['Dogum Yeri', 'Ikametgah Sehri', 'Lise Sehir']:
    train_df[col] = train_df[col].astype(str).apply(process_city_column)

In [13]:
train_df['Dogum Yeri'].value_counts().head(20)

,count
Dogum Yeri,
istanbul,10922
ankara,4223
izmir,2822
bursa,1902
adana,1890
diyarbakir,1635
mersin,1491
konya,1489
gaziantep,1359


In [14]:
# Function for processing 'Bölüm' column
def process_bolum_column(text):
    text = text.replace('muhendisligi', 'muh')
    text = text.replace('-', ' ')

    # Strip white spaces
    return text.strip()

train_df['Bölüm'] = train_df['Bölüm'].astype(str).apply(process_bolum_column)

In [15]:
train_df['Bölüm'].value_counts().head(20)

,count
Bölüm,
isletme,2913
endustri muh,2838
bilgisayar muh,2249
hukuk,2007
elektrik elektronik muh,1797
makine muh,1557
iktisat,1529
tip,1334
insaat muh,1209


In [16]:
from datetime import datetime

train_df['Dogum Tarihi'] = pd.to_datetime(train_df['Dogum Tarihi'], errors='coerce')
current_date = datetime.now()

# Calculate age by finding the difference between the current date and 'Dogum Tarihi'
train_df['Age'] = train_df['Dogum Tarihi'].apply(lambda x: current_date.year - x.year - ((current_date.month, current_date.day) < (x.month, x.day)) if pd.notnull(x) else np.nan)

# fill missing with median age
median_age = train_df['Age'].median()
train_df['Age'].fillna(median_age, inplace=True)

# Drop the 'Dogum Tarihi' column
train_df.drop(columns=['Dogum Tarihi'], inplace=True)

In [17]:
train_df['Age'].value_counts()

,count
Age,
28.0,48708
27.0,3218
26.0,2673
29.0,2604
30.0,2059
31.0,1582
25.0,1315
32.0,1146
24.0,636


In [18]:
education_mapping = {
    'yuksek lisans / doktora': 5,
    'yuksek lisans / doktara': 5,
    'yuksek lisans': 5,
    'doktora': 5,
    'ilkokul mezunu': 1,
    'ilkokul': 1,
    'ilk okul': 1,
    'ortaokul mezunu': 2,
    'ortaokul': 2,
    'orta okul': 2,
    'lise mezunu': 3,
    'lise': 3,
    'liseli' : 3,
    'universite mezunu': 4,
    'universite': 4,
    'mezun': 4,
    'egitim yok': 0,
    'egitimi yok': 0
}

# Define the mapping for 'Calisma Durumu'
calisma_mapping = {
    'evet': 1,
    'hayir': 0,
    'emekli': 2
}

for col in ['Anne Calisma Durumu', 'Baba Calisma Durumu']:
    train_df[col] = train_df[col].replace(calisma_mapping)
    train_df[col].fillna(0, inplace=True)


# Apply the mapping to both columns
for col in ['Anne Egitim Durumu', 'Baba Egitim Durumu']:
    train_df[col] = train_df[col].str.lower().replace(education_mapping)
    train_df[col].fillna(0, inplace=True)

In [19]:
train_df['Baba Egitim Durumu'].value_counts()

,count
Baba Egitim Durumu,
1,19429
3,15895
4,15138
2,9180
0,3505
5,1866
0,111


In [20]:
train_df['Baba Calisma Durumu'].value_counts()

,count
Baba Calisma Durumu,
1.0,28558
0.0,25104
2.0,11462


In [21]:
# Define a mapping for class years and special categories
class_mapping = {
    'hazirlik': 0,
    'mezun': 7,
    'yuksek lisans': 7,
    'tez': 7,
    '0': -1  # Handle unexpected/unknown values
}

# Standardize and map the values for 'Universite Kacinci Sinif'
def standardize_class(value):
    if pd.isna(value) or value in ['nan', '']:  # Check for NaN and empty string
        return -1  # Assign a placeholder value for missing
    try:
        value = int(value)
        if 1 <= value <= 6:
            return value
    except ValueError:
        return class_mapping.get(value.lower(), -1)

# Apply the function to 'Universite Kacinci Sinif'
train_df['Universite Kacinci Sinif'] = train_df['Universite Kacinci Sinif'].apply(standardize_class)

# Handle missing values (fill '-1' with mode)
mode_value = train_df['Universite Kacinci Sinif'][train_df['Universite Kacinci Sinif'] != -1].mode()[0]
train_df['Universite Kacinci Sinif'].replace(-1, mode_value, inplace=True)

# Fill any remaining NaN values explicitly, if any
train_df['Universite Kacinci Sinif'].fillna(mode_value, inplace=True)

In [22]:
train_df['Universite Kacinci Sinif'].isnull().sum()

0

In [23]:
train_df['Universite Kacinci Sinif'].value_counts()

,count
Universite Kacinci Sinif,
2.0,22150
3.0,16956
4.0,13112
1.0,10260
0.0,1308
5.0,961
7.0,200
6.0,177


In [24]:
# Convert the column to string and filter non-numeric rows
train_df['Kardes Sayisi'].isnull().sum()

1132

In [25]:
# Convert the column back to string to clean non-numeric values
train_df['Kardes Sayisi'] = train_df['Kardes Sayisi'].astype(str)

# Remove non-numeric rows, setting them to NaN
train_df['Kardes Sayisi'] = pd.to_numeric(train_df['Kardes Sayisi'], errors='coerce')

# convert float to int
train_df['Kardes Sayisi'] = train_df['Kardes Sayisi'].fillna(-1)
train_df['Kardes Sayisi'] = train_df['Kardes Sayisi'].astype(int)

# Get the top 3 most frequent values
top_3_values = train_df['Kardes Sayisi'].value_counts().nlargest(3).index.tolist()

#fill missing with top 3 values randomly
train_df.loc[train_df['Kardes Sayisi'] == -1, 'Kardes Sayisi'] = np.random.choice(top_3_values, size=(train_df['Kardes Sayisi'] == -1).sum())

print(train_df['Kardes Sayisi'].value_counts())

Kardes Sayisi
1     22637
2     17369
3      9187
4      5609
0      5082
5      4136
6       341
7       278
8       173
10      169
9       122
11       12
12        5
13        2
18        1
14        1
Name: count, dtype: int64


In [26]:
# Create a mapping function to convert ranges and handle non-numeric values
def convert_grade_range(grade):
    # Handle NaN values (which are floats)
    if pd.isna(grade):
        return np.nan
    # Ensure grade is a string for further processing
    grade = str(grade)

    # Handle non-numeric categories
    if grade in ['ortalama bulunmuyor', 'not ortalamasi yok', 'hazirligim']:
        return np.nan
    # Handle ranges by calculating their midpoints
    elif '-' in grade:
        range_values = grade.replace('ve alti', '-').replace('ve ustu', '-').split('-')
        range_values = [float(val) for val in range_values if val.strip()]
        return np.mean(range_values)
    else:
        return np.nan

# Apply the conversion function to the column
train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].apply(convert_grade_range)

# Get the non-NaN values
non_nan_values = train_df['Universite Not Ortalamasi'].dropna().values

# Fill the NaN values with random selections from the non-NaN values
train_df['Universite Not Ortalamasi'] = train_df['Universite Not Ortalamasi'].apply(
    lambda x: np.random.choice(non_nan_values) if np.isnan(x) else x
)

# Check the final result
print(train_df['Universite Not Ortalamasi'].value_counts())

Universite Not Ortalamasi
2.750    24878
3.250    19284
3.750     6710
2.250     3449
2.745     3338
3.245     3244
3.500     1939
2.145     1594
1.750      441
0.895      247
Name: count, dtype: int64


In [27]:
# Define a mapping for normalizing similar categories
lise_turu_mapping = {
    'anadolu lisesi': 'devlet',
    'diger': 'diger',
    'duz lise': 'devlet',
    'devlet': 'devlet',
    'ozel': 'ozel',  # Combine ozel, ozel lise, and ozel lisesi
    'ozel lisesi': 'ozel',
    'ozel lise': 'ozel',
    'meslek lisesi': 'devlet',
    'meslek': 'devlet',  # Combine meslek and meslek lisesi
    'fen lisesi': 'devlet',
    'imam hatip lisesi': 'devlet',
}

# Apply the mapping to the 'Lise Turu' column
train_df['Lise Turu'] = train_df['Lise Turu'].str.lower().map(lise_turu_mapping)

# Get unique categories from the mapping
categories = list(set(lise_turu_mapping.values()))

# Fill missing values with random choices from the categories
train_df['Lise Turu'] = train_df['Lise Turu'].apply(
    lambda x: np.random.choice(categories) if pd.isna(x) else x
)

In [28]:
train_df['Lise Turu'].value_counts()

,count
Lise Turu,
devlet,44341
diger,13453
ozel,7330


In [29]:
# Check for missing values
print(train_df.isnull().sum().sort_values(ascending=True))

Basvuru Yili                                                0
Ingilizce Biliyor musunuz?                                  0
Girisimcilikle Ilgili Deneyiminiz Var Mi?                   0
Stk Projesine Katildiniz Mi?                                0
Aktif olarak bir STK üyesi misiniz?                         0
Profesyonel Bir Spor Daliyla Mesgul musunuz?                0
Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?        0
Kardes Sayisi                                               0
Baba Calisma Durumu                                         0
Baba Egitim Durumu                                          0
Anne Calisma Durumu                                         0
Anne Egitim Durumu                                          0
id                                                          0
Lise Turu                                                   0
Baska Bir Kurumdan Burs Aliyor mu?                          0
Burs Aliyor mu?                                             0
Degerlen

In [30]:
train_df['Lise Bolumu'].value_counts().head(30)

,count
Lise Bolumu,
sayisal,29639
esit agirlik,17420
fen sayisal bilimleri alani,3995
sozel,3697
esit agirlik turkce-matematik alani,2786
dil,784
diger,743
tm,717
yabanci dil,532


In [31]:
sayisal_keywords = ['sayisal', 'fen', 'matematik', 'mf', 'fen sayisal bilimleri', 'fen matematik', 'matematik fen']
esit_agirlik_keywords = ['esitagirlik', 'esit agirlik', 'tm', 'turkce matematik','turkce  matematik', 'ea']
sozel_keywords = ['sozel', 'sosyal', 'ts', 'sosyal bilimler']
dil_keywords = ['dil', 'yabanci dil']

def map_lise_bolumu(value):
    value = str(value).lower()
    value = re.sub(r'[-/]', ' ', value)

    if any(keyword in value for keyword in sayisal_keywords):
        return 'sayisal'
    elif any(keyword in value for keyword in esit_agirlik_keywords):
        return 'esit agirlik'
    elif any(keyword in value for keyword in sozel_keywords):
        return 'sozel'
    elif any(keyword in value for keyword in dil_keywords):
        return 'dil'
    else:
        return 'diger'

# Apply the function to the 'Lise Bolumu' column
train_df['Lise Bolumu'] = train_df['Lise Bolumu'].apply(map_lise_bolumu)

In [32]:
train_df['Lise Bolumu'].value_counts().head(30)

,count
Lise Bolumu,
sayisal,38434
esit agirlik,18264
sozel,4239
diger,2675
dil,1512
